# Initialization and Optimization

For this lab on initialization and optimization, let's look at a slightly different type of neural network. This time, we will not perform a classification task as we've done before.  Instead, we'll look at a regression problem.

We can just as well use deep learning networks for regression as for a classification problem. However, note that getting regression to work with neural networks is a harder problem because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$, and the models are especially prone to **_exploding gradients_**. This issue makes a regression exercise the perfect learning case!

Run the cell below to import everything we'll need for this lab.

In [2]:
import numpy as np
np.random.seed(0)
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from keras import optimizers
from sklearn.model_selection import train_test_split

## 1. Loading the data

The data we'll be working with is data related to facebook posts published during the year of 2014 on the Facebook's page of a renowned cosmetics brand.  It includes 7 features known prior to post publication, and 12 features for evaluating the post impact. What we want to do is make a predictor for the number of "likes" for a post, taking into account the 7 features prior to posting.

First, let's import the data set and delete any rows with missing data.  

The dataset is contained with the file `dataset_Facebook.csv`. In the cell below, use pandas to read in the data from this file. Because of the way the data is structure, make sure you also set the `sep` parameter to `";"`, and the `header` parameter to `0`. 

Then, use the DataFrame's built-in `.dropna()` function to remove any rows with missing values. 

In [7]:
# load dataset
data = pd.read_csv('dataset_Facebook.csv', sep=';', header=0)
data = data.dropna()

Now, let's check the shape of our data to ensure that everything looks correct. 

In [8]:
np.shape(data) #Expected Output: (495, 19)

(495, 19)

And finally, let's inspect the `.head()` of the DataFrame to get a feel for what our dataset looks like. 

In [11]:
data.head()

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


## 2. Initialization

### 2.1 Normalize the input data

A big part of Deep Learning is cleaning the data and getting into a shape usable by a neural network.  Let's get some additional practice with this.


Take a look at our input data. We'll use the 7 first columns as our predictors. We'll do the following two things:
- Normalize the continuous variables --> you can do this using `np.mean()` and `np.std()`
- make dummy variables of the categorical variables (you can do this by using `pd.get_dummies`)

We only count "Category" and "Type" as categorical variables. Note that you can argue that "Post month", "Post Weekday" and "Post Hour" can also be considered categories, but we'll just treat them as being continuous for now.

In the cell below, convert the data as needed by normalizing or converting to dummy variables, and then concatenate it all back into a single DataFrame once you've finished.  

In [18]:
X0 = data["Page total likes"]
X1 = data["Type"]
X2 = data["Category"]
X3 = data["Post Month"]
X4 = data["Post Weekday"]
X5 = data["Post Hour"]
X6 = data["Paid"]

## standardize/categorize
X0= (X0-np.mean(X0))/np.std(X0)
dummy_X1= pd.get_dummies(X1)
dummy_X2= pd.get_dummies(X2)
X3= (X3-np.mean(X3))/np.std(X3)
X4= (X4-np.mean(X4))/np.std(X4)
X5= (X5-np.mean(X5))/np.std(X5)

# Add them all back into a single DataFrame
X = pd.concat([X0, dummy_X1, dummy_X2, X3, X4, X5, X6], axis=1)

# Store our labels in a separate variable
Y = data["like"]

In [20]:
#Note: you get the same result for standardization if you use StandardScaler from sklearn.preprocessing

# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X0 = sc.fit_transform(X0)

Our data is fairly small. Let's just split the data up in a training set and a validation set!

In the cell below:

* Split the data into training and testing sets by passing `X` and `Y` into `train_test_split`.  Set a `test_size` of `0.2`.

In [21]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)

Let's check the shape to make sure everything worked correctly.

In [22]:
X_val.shape # Expected Output: (99, 12)

(99, 12)

In [23]:
X_train.shape # Expected Output: (396, 12)

(396, 12)

## Building a Neural Network for Regression

Now, we'll build a neural network to predict the number of likes we think a post will receive.  

In the cell below, create a model with the following specifications:

* 1 Hidden Layer with 8 neurons.  In this layer, also set `input_dim` to `12`, and `activation` to `"relu"`.
* An output layer with 1 neuron.  For this neuron, set the activation to `linear`.  

In [24]:
model = Sequential()

model.add(Dense(8, activation='relu', input_dim=12))
model.add(Dense(1, activation='linear'))

Now, we need to compile the model, with the following hyperparameters:

* `optimizer='sgd'`
* `loss='mse'`
* `metrics=['mse']`

Note that since our model is training for a regression task, not a classification task, we'll need to use a loss metric that corresponds with regression tasks--Mean Squared Error. 

In [25]:
model.compile(optimizer='SGD', loss='MSE', metrics=['MSE'])

Finally, let's train the model.  Call `model.fit()`. In addition to to the training data and labels, also set:

* `batch_size=32`
* `epochs=100`
* `verbose=1`
* `validation_data=(X_val, y_val)`

In [26]:
hist = model.fit(X_train, Y_train, batch_size=32, epochs=100,
                verbose=1, validation_data=(X_val, Y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 519us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/100
396/396 [==============================] - 0s 48us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/100
396/396 [==============================] - 0s 60us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/100
396/396 [==============================] - 0s 89us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/100
396/396 [==============================] - 0s 64us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/100
396/396 [==============================] - 0s 78us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/100
396/396 [=====================

396/396 [==============================] - 0s 54us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/100
396/396 [==============================] - 0s 65us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/100
396/396 [==============================] - 0s 66us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/100
396/396 [==============================] - 0s 50us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/100
396/396 [==============================] - 0s 49us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/100
396/396 [==============================] - 0s 45us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 60/100
396/396 [==============================] - 0s 44us/step - loss: nan - mean_squared

Did you see what happend? all the values for training and validation loss are "nan". There could be several reasons for that, but as we already mentioned there is likely a vanishing or exploding gradient problem.  This means that the values got so large or so small that they no longer fit in memory.   R

Recall that we normalized out inputs. But how about the outputs? Let's have a look.

In [27]:
Y_train.head()

212     36.0
107    193.0
411     75.0
71     449.0
473    136.0
Name: like, dtype: float64

Yes, indeed. We didn't normalize them and we should, as they take pretty high values. Let
s rerun the model but make sure that the output is normalized as well!

### 2.2 Normalizing the output

In the cell below, we've included all the normalization code that we wrote up top, but this time, we've added a line to normalize the data in `Y`, as well. This should help alot!

Run the cell below to normalize our data and our labels.

In [28]:
X0 = data["Page total likes"]
X1 = data["Type"]
X2 = data["Category"]
X3 = data["Post Month"]
X4 = data["Post Weekday"]
X5 = data["Post Hour"]
X6 = data["Paid"]

## standardize/categorize
X0= (X0-np.mean(X0))/(np.std(X0))
dummy_X1= pd.get_dummies(X1)
dummy_X2= pd.get_dummies(X2)
X3= (X3-np.mean(X3))/(np.std(X3))
X4= (X4-np.mean(X4))/(np.std(X4))
X5= (X5-np.mean(X5))/(np.std(X5))

X = pd.concat([X0, dummy_X1, dummy_X2, X3, X4, X5, X6], axis=1)

Y = (data["like"]-np.mean(data["like"]))/(np.std(data["like"]))

Now, let's split our data into appropriate training and testing sets again.  Split the data, just like we did before.  Use the same `test_size` as we did last time, too. 

In [29]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)

Now, let's reinitialize our model and build it from scratch again.  

**_NOTE:_**  If we don't reinitialize our model, our training would start with the weight values we ended with during the last training session.  In order to start fresh, we need to declare a new `Sequential()` object.  

Build the model with the exact same architecture and hyperparameters as we did above.

In [30]:
model = Sequential()

model.add(Dense(8, activation='relu', input_dim=12))
model.add(Dense(1, activation='linear'))

Now, compile the model with the same parameters we used before.

In [31]:
model.compile(optimizer='SGD', loss='mse', metrics=['mse'])

And finally, fit the model using the same parameters we did before. 

In [32]:
hist = model.fit(X_train, Y_train, batch_size=32, epochs=100,
                verbose=1, validation_data=(X_val, Y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 516us/step - loss: 1.7555 - mean_squared_error: 1.7555 - val_loss: 0.3350 - val_mean_squared_error: 0.3350
Epoch 2/100
396/396 [==============================] - 0s 49us/step - loss: 1.3417 - mean_squared_error: 1.3417 - val_loss: 0.3002 - val_mean_squared_error: 0.3002
Epoch 3/100
396/396 [==============================] - 0s 65us/step - loss: 1.2700 - mean_squared_error: 1.2700 - val_loss: 0.3005 - val_mean_squared_error: 0.3005
Epoch 4/100
396/396 [==============================] - 0s 71us/step - loss: 1.2396 - mean_squared_error: 1.2396 - val_loss: 0.2923 - val_mean_squared_error: 0.2923
Epoch 5/100
396/396 [==============================] - 0s 76us/step - loss: 1.2186 - mean_squared_error: 1.2186 - val_loss: 0.2870 - val_mean_squared_error: 0.2870
Epoch 6/100
396/396 [==============================] - 0s 96us/step - loss: 1.2036 - mean_squared_error: 1.2036 - val_loss: 0.2840 - v

396/396 [==============================] - 0s 46us/step - loss: 1.0906 - mean_squared_error: 1.0906 - val_loss: 0.2579 - val_mean_squared_error: 0.2579
Epoch 51/100
396/396 [==============================] - 0s 45us/step - loss: 1.0906 - mean_squared_error: 1.0906 - val_loss: 0.2595 - val_mean_squared_error: 0.2595
Epoch 52/100
396/396 [==============================] - 0s 65us/step - loss: 1.0878 - mean_squared_error: 1.0878 - val_loss: 0.2539 - val_mean_squared_error: 0.2539
Epoch 53/100
396/396 [==============================] - 0s 87us/step - loss: 1.0873 - mean_squared_error: 1.0873 - val_loss: 0.2488 - val_mean_squared_error: 0.2488
Epoch 54/100
396/396 [==============================] - 0s 91us/step - loss: 1.0885 - mean_squared_error: 1.0885 - val_loss: 0.2562 - val_mean_squared_error: 0.2562
Epoch 55/100
396/396 [==============================] - 0s 48us/step - loss: 1.0860 - mean_squared_error: 1.0860 - val_loss: 0.2616 - val_mean_squared_error: 0.2616
Epoch 56/100
396/396 [=

Epoch 100/100
396/396 [==============================] - 0s 75us/step - loss: 1.0576 - mean_squared_error: 1.0576 - val_loss: 0.2832 - val_mean_squared_error: 0.2832


The model did much, much better this time around!

Run the cell below to get the model's predictions for both the training and validation sets. 

In [33]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

Let's look at the first 10 predictions from `pred_train`. Display those in the cell below.

In [34]:
pred_train[:10]

array([ 0.1336047 , -0.48186985, -0.30847377,  0.5457481 , -0.0837617 ,
       -0.05147585,  0.10491547,  0.10348544, -0.4340665 , -0.06438229],
      dtype=float32)

Let's manually calculate the Mean Squared Error in the cell below.  

As a refresher, here's the formula for calculating Mean Squared Error:

<img src='mse_formula.gif'>

Use `pred_train` and `Y_train` to calculate our training MSE in the cell below.  

**_HINT:_** Use numpy to make short work of this!

In [36]:
MSE_train = np.sum(np.power((pred_train-Y_train),2))/len(pred_train)
MSE_train 

1.0534526639414505

Now, calculate the MSE for our validation set in the cell below.

In [37]:
MSE_val = np.mean((pred_val-Y_val)**2)
MSE_val 

0.28320759811935026

### 2.3 Use weight initializers

Another way to increase the performance of our models is to initialize our weights in clever ways.  We'll explore some of those options below.  

#### 2.3.1  He initialization

Let's try and use a weight initializer.  We'll start with the **_He normalizer_**, which initializes the weight vector to have an average 0 and a variance of 2/n, with $n$ the number of features feeding into a layer.

In the cell below:

* Recreate the Neural Network that we created above.  This time, in the hidden layer, set the `kernel_initializer` to `"he_normal"`.
* Compile and fit the model with the same hyperparameters as we used before.  

In [38]:
model = Sequential()

model.add(Dense(8, activation='relu', input_dim=12, kernel_initializer='he_normal'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='sgd', loss='mse', metrics=['mse'])

he_model = model.fit(X_train, Y_train, batch_size=32, epochs=100,
                    verbose=1, validation_data=(X_val, Y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 556us/step - loss: 3.0270 - mean_squared_error: 3.0270 - val_loss: 0.8771 - val_mean_squared_error: 0.8771
Epoch 2/100
396/396 [==============================] - 0s 63us/step - loss: 1.7447 - mean_squared_error: 1.7447 - val_loss: 0.5735 - val_mean_squared_error: 0.5735
Epoch 3/100
396/396 [==============================] - 0s 81us/step - loss: 1.4373 - mean_squared_error: 1.4373 - val_loss: 0.4822 - val_mean_squared_error: 0.4822
Epoch 4/100
396/396 [==============================] - 0s 72us/step - loss: 1.3241 - mean_squared_error: 1.3241 - val_loss: 0.3776 - val_mean_squared_error: 0.3776
Epoch 5/100
396/396 [==============================] - 0s 82us/step - loss: 1.2773 - mean_squared_error: 1.2773 - val_loss: 0.3714 - val_mean_squared_error: 0.3714
Epoch 6/100
396/396 [==============================] - 0s 66us/step - loss: 1.2469 - mean_squared_error: 1.2469 - val_loss: 0.3290 - v

396/396 [==============================] - 0s 71us/step - loss: 1.1251 - mean_squared_error: 1.1251 - val_loss: 0.2907 - val_mean_squared_error: 0.2907
Epoch 51/100
396/396 [==============================] - 0s 71us/step - loss: 1.1263 - mean_squared_error: 1.1263 - val_loss: 0.2797 - val_mean_squared_error: 0.2797
Epoch 52/100
396/396 [==============================] - 0s 60us/step - loss: 1.1261 - mean_squared_error: 1.1261 - val_loss: 0.2785 - val_mean_squared_error: 0.2785
Epoch 53/100
396/396 [==============================] - 0s 44us/step - loss: 1.1231 - mean_squared_error: 1.1231 - val_loss: 0.2752 - val_mean_squared_error: 0.2752
Epoch 54/100
396/396 [==============================] - 0s 79us/step - loss: 1.1223 - mean_squared_error: 1.1223 - val_loss: 0.2941 - val_mean_squared_error: 0.2941
Epoch 55/100
396/396 [==============================] - 0s 53us/step - loss: 1.1219 - mean_squared_error: 1.1219 - val_loss: 0.2821 - val_mean_squared_error: 0.2821
Epoch 56/100
396/396 [=

Epoch 100/100
396/396 [==============================] - 0s 66us/step - loss: 1.1039 - mean_squared_error: 1.1039 - val_loss: 0.2897 - val_mean_squared_error: 0.2897


Great!

Run the cells below to get training and validation predictions are recalculate our MSE for each. 

In [39]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [40]:
print(MSE_train) 
print(MSE_val) 

1.0980026908011942
0.2896783150613855


The initializer does not really help us to decrease the MSE. We know that initializers can be particularly helpful in deeper networks, and our network isn't very deep. What if we use the `Lecun` initializer with a `tanh` activation?

#### 2.3.2  Lecun initialization

In the cell below, recreate the network again.  This time, set hidden layer's activation to `'tanh'`, and the `kernel_initializer` to `'lecun_normal'`.

Then, fit and compile the model as did before.  

In [42]:
model = Sequential()

model.add(Dense(8, activation='tanh', input_dim=12, kernel_initializer='lecun_normal'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='sgd', loss='mse', metrics=['mse'])

lecun_model = model.fit(X_train, Y_train, batch_size=32, epochs=100,
                        verbose=1, validation_data=(X_val, Y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 620us/step - loss: 1.5673 - mean_squared_error: 1.5673 - val_loss: 0.3069 - val_mean_squared_error: 0.3069
Epoch 2/100
396/396 [==============================] - 0s 55us/step - loss: 1.3487 - mean_squared_error: 1.3487 - val_loss: 0.2724 - val_mean_squared_error: 0.2724
Epoch 3/100
396/396 [==============================] - 0s 82us/step - loss: 1.2831 - mean_squared_error: 1.2831 - val_loss: 0.2597 - val_mean_squared_error: 0.2597
Epoch 4/100
396/396 [==============================] - 0s 125us/step - loss: 1.2483 - mean_squared_error: 1.2483 - val_loss: 0.2610 - val_mean_squared_error: 0.2610
Epoch 5/100
396/396 [==============================] - 0s 92us/step - loss: 1.2300 - mean_squared_error: 1.2300 - val_loss: 0.2533 - val_mean_squared_error: 0.2533
Epoch 6/100
396/396 [==============================] - 0s 75us/step - loss: 1.2140 - mean_squared_error: 1.2140 - val_loss: 0.2574 - 

396/396 [==============================] - 0s 142us/step - loss: 1.1318 - mean_squared_error: 1.1318 - val_loss: 0.2608 - val_mean_squared_error: 0.2608
Epoch 51/100
396/396 [==============================] - 0s 53us/step - loss: 1.1327 - mean_squared_error: 1.1327 - val_loss: 0.2577 - val_mean_squared_error: 0.2577
Epoch 52/100
396/396 [==============================] - 0s 45us/step - loss: 1.1309 - mean_squared_error: 1.1309 - val_loss: 0.2589 - val_mean_squared_error: 0.2589
Epoch 53/100
396/396 [==============================] - 0s 59us/step - loss: 1.1306 - mean_squared_error: 1.1306 - val_loss: 0.2594 - val_mean_squared_error: 0.2594
Epoch 54/100
396/396 [==============================] - 0s 55us/step - loss: 1.1320 - mean_squared_error: 1.1320 - val_loss: 0.2547 - val_mean_squared_error: 0.2547
Epoch 55/100
396/396 [==============================] - 0s 86us/step - loss: 1.1321 - mean_squared_error: 1.1321 - val_loss: 0.2557 - val_mean_squared_error: 0.2557
Epoch 56/100
396/396 [

396/396 [==============================] - 0s 50us/step - loss: 1.1207 - mean_squared_error: 1.1207 - val_loss: 0.2492 - val_mean_squared_error: 0.2492
Epoch 100/100
396/396 [==============================] - 0s 48us/step - loss: 1.1204 - mean_squared_error: 1.1204 - val_loss: 0.2580 - val_mean_squared_error: 0.2580


Now, run the cells below to get the predictions and calculate the MSE for training and validation again.  

In [43]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [44]:
print(MSE_train) 
print(MSE_val) 

1.1152763443738078
0.2579843313651145


## 3. Optimization

Another option we have is to play with the optimizers we choose for gradient descent during our back propagation step.  So far, we've only made use of basic `'sgd'`, or **_Stochastic Gradient Descent_**.  However, there are more advanced optimizers available to use will often converge to better minima, usually in a quicker fashion. 

In this lab, we'll try the two most popular methods: **_RMSprop_** and **_adam_**.

### 3.1 RMSprop

In the cell below, recreate the original network that we built in this lab--no kernel intialization parameter, and the activation set to `'relu'`. 

This time, when you compile the model, set the `optimizer` parameter to `"rmsprop"`.  No changes to the `fit()` call are needed--keep those parameters the same.  

In [45]:
model = Sequential()

model.add(Dense(8, activation='relu', input_dim=12))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])

rms_model = model.fit(X_train, Y_train, batch_size=32, epochs=100,
                      verbose=1, validation_data=(X_val, Y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 857us/step - loss: 1.2056 - mean_squared_error: 1.2056 - val_loss: 0.2944 - val_mean_squared_error: 0.2944
Epoch 2/100
396/396 [==============================] - 0s 129us/step - loss: 1.1889 - mean_squared_error: 1.1889 - val_loss: 0.2942 - val_mean_squared_error: 0.2942
Epoch 3/100
396/396 [==============================] - 0s 89us/step - loss: 1.1784 - mean_squared_error: 1.1784 - val_loss: 0.2957 - val_mean_squared_error: 0.2957
Epoch 4/100
396/396 [==============================] - 0s 57us/step - loss: 1.1703 - mean_squared_error: 1.1703 - val_loss: 0.2949 - val_mean_squared_error: 0.2949
Epoch 5/100
396/396 [==============================] - 0s 79us/step - loss: 1.1649 - mean_squared_error: 1.1649 - val_loss: 0.2938 - val_mean_squared_error: 0.2938
Epoch 6/100
396/396 [==============================] - 0s 114us/step - loss: 1.1587 - mean_squared_error: 1.1587 - val_loss: 0.2923 -

396/396 [==============================] - 0s 140us/step - loss: 1.0835 - mean_squared_error: 1.0835 - val_loss: 0.2891 - val_mean_squared_error: 0.2891
Epoch 51/100
396/396 [==============================] - 0s 73us/step - loss: 1.0815 - mean_squared_error: 1.0815 - val_loss: 0.2867 - val_mean_squared_error: 0.2867
Epoch 52/100
396/396 [==============================] - 0s 64us/step - loss: 1.0810 - mean_squared_error: 1.0810 - val_loss: 0.2846 - val_mean_squared_error: 0.2846
Epoch 53/100
396/396 [==============================] - 0s 64us/step - loss: 1.0802 - mean_squared_error: 1.0802 - val_loss: 0.2858 - val_mean_squared_error: 0.2858
Epoch 54/100
396/396 [==============================] - 0s 48us/step - loss: 1.0781 - mean_squared_error: 1.0781 - val_loss: 0.2887 - val_mean_squared_error: 0.2887
Epoch 55/100
396/396 [==============================] - 0s 78us/step - loss: 1.0769 - mean_squared_error: 1.0769 - val_loss: 0.2855 - val_mean_squared_error: 0.2855
Epoch 56/100
396/396 [

Epoch 100/100
396/396 [==============================] - 0s 76us/step - loss: 1.0430 - mean_squared_error: 1.0430 - val_loss: 0.2957 - val_mean_squared_error: 0.2957


Now, run the cell below to get predictions and compute the MSE again.

In [46]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [47]:
print(MSE_train) 
print(MSE_val) 

1.040166996711317
0.29571725642903124


### 3.2 Adam

Another popular optimizer is **_adam_**, which stands for `Adaptive Moment Estimation`. This is an optimzer that was created and open-sourced by a team at OpenAI, and is generally seen as the go-to choice for optimizers today. Adam combines the RMSprop algorithm with the concept of momentum, and is generally very effective at getting converging quickly.  

In the cell below, create the same network that we did above, but this time, set the optimizer parameter to `'adam'`.  Leave all other parameters the same. 

In [48]:
model = Sequential()

model.add(Dense(8, activation='relu', input_dim=12))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mse', metrics=['mse'])

adam_model = model.fit(X_train, Y_train, batch_size=32, epochs=100,
                      verbose=1, validation_data=(X_val, Y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 978us/step - loss: 1.5154 - mean_squared_error: 1.5154 - val_loss: 0.6113 - val_mean_squared_error: 0.6113
Epoch 2/100
396/396 [==============================] - 0s 122us/step - loss: 1.4299 - mean_squared_error: 1.4299 - val_loss: 0.5200 - val_mean_squared_error: 0.5200
Epoch 3/100
396/396 [==============================] - 0s 60us/step - loss: 1.3631 - mean_squared_error: 1.3631 - val_loss: 0.4620 - val_mean_squared_error: 0.4620
Epoch 4/100
396/396 [==============================] - 0s 56us/step - loss: 1.3189 - mean_squared_error: 1.3189 - val_loss: 0.4145 - val_mean_squared_error: 0.4145
Epoch 5/100
396/396 [==============================] - 0s 91us/step - loss: 1.2829 - mean_squared_error: 1.2829 - val_loss: 0.3751 - val_mean_squared_error: 0.3751
Epoch 6/100
396/396 [==============================] - 0s 74us/step - loss: 1.2564 - mean_squared_error: 1.2564 - val_loss: 0.3393 - 

396/396 [==============================] - 0s 60us/step - loss: 1.1200 - mean_squared_error: 1.1200 - val_loss: 0.2371 - val_mean_squared_error: 0.2371
Epoch 51/100
396/396 [==============================] - 0s 97us/step - loss: 1.1192 - mean_squared_error: 1.1192 - val_loss: 0.2345 - val_mean_squared_error: 0.2345
Epoch 52/100
396/396 [==============================] - 0s 77us/step - loss: 1.1188 - mean_squared_error: 1.1188 - val_loss: 0.2398 - val_mean_squared_error: 0.2398
Epoch 53/100
396/396 [==============================] - 0s 58us/step - loss: 1.1169 - mean_squared_error: 1.1169 - val_loss: 0.2361 - val_mean_squared_error: 0.2361
Epoch 54/100
396/396 [==============================] - 0s 118us/step - loss: 1.1154 - mean_squared_error: 1.1154 - val_loss: 0.2385 - val_mean_squared_error: 0.2385
Epoch 55/100
396/396 [==============================] - 0s 202us/step - loss: 1.1148 - mean_squared_error: 1.1148 - val_loss: 0.2364 - val_mean_squared_error: 0.2364
Epoch 56/100
396/396 

396/396 [==============================] - 0s 86us/step - loss: 1.0799 - mean_squared_error: 1.0799 - val_loss: 0.2590 - val_mean_squared_error: 0.2590
Epoch 100/100
396/396 [==============================] - 0s 73us/step - loss: 1.0797 - mean_squared_error: 1.0797 - val_loss: 0.2530 - val_mean_squared_error: 0.2530


In [49]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [50]:
print(MSE_train) 
print(MSE_val) 

1.0780708940958597
0.25298893950622975


### 3.3 Learning rate decay with momentum


The final item that we'll get practice with in this lab is implementing a **_Learning Rate Decay_** strategy, along with **_Momentum_**.  We'll accomplish this by creating a `SGD` object and setting learning rate, decay, and momentum parameters at initialization.  In this way, we can then pass in the `SGD` object we've initialized to our specificataions during the compile step, rather than just a string representing an off-the-shelf `'SGD'`  optimizer.  

In the cell below:

* Create a `SGD` optimizer, which can be found in the `optimizers` module.  
    * Set the `lr` parameter to  `0.03`.
    * Set the `decay` parameter to `0.0001`
    * Set the `momentum` parameter to `0.9`.
* Recreate the same network we used during the previous example.  
* Set the optimizer parameter during the compile step to the `sgd` object we created. 
* Fit the model with the same hyperparameters as we used before.  

In [51]:
sgd = optimizers.SGD(lr=0.03, decay=0.0001, momentum=0.9)

model = Sequential()

model.add(Dense(8, activation='relu', input_dim=12))
model.add(Dense(1, activation='linear'))

model.compile(optimizer=sgd, loss='mse', metrics=['mse'])

decay_model = model.fit(X_train, Y_train, batch_size=32, epochs=100,
                      verbose=1, validation_data=(X_val, Y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 907us/step - loss: 1.5011 - mean_squared_error: 1.5011 - val_loss: 0.5075 - val_mean_squared_error: 0.5075
Epoch 2/100
396/396 [==============================] - 0s 116us/step - loss: 1.2213 - mean_squared_error: 1.2213 - val_loss: 0.2190 - val_mean_squared_error: 0.2190
Epoch 3/100
396/396 [==============================] - 0s 72us/step - loss: 1.1833 - mean_squared_error: 1.1833 - val_loss: 0.3215 - val_mean_squared_error: 0.3215
Epoch 4/100
396/396 [==============================] - 0s 67us/step - loss: 1.2030 - mean_squared_error: 1.2030 - val_loss: 0.2034 - val_mean_squared_error: 0.2034
Epoch 5/100
396/396 [==============================] - 0s 66us/step - loss: 1.1737 - mean_squared_error: 1.1737 - val_loss: 0.2153 - val_mean_squared_error: 0.2153
Epoch 6/100
396/396 [==============================] - 0s 52us/step - loss: 1.1575 - mean_squared_error: 1.1575 - val_loss: 0.2509 - 

396/396 [==============================] - 0s 83us/step - loss: 1.0552 - mean_squared_error: 1.0552 - val_loss: 0.3036 - val_mean_squared_error: 0.3036
Epoch 51/100
396/396 [==============================] - 0s 90us/step - loss: 1.0245 - mean_squared_error: 1.0245 - val_loss: 0.3741 - val_mean_squared_error: 0.3741
Epoch 52/100
396/396 [==============================] - 0s 57us/step - loss: 1.0336 - mean_squared_error: 1.0336 - val_loss: 0.3547 - val_mean_squared_error: 0.3547
Epoch 53/100
396/396 [==============================] - 0s 89us/step - loss: 1.0380 - mean_squared_error: 1.0380 - val_loss: 0.3544 - val_mean_squared_error: 0.3544
Epoch 54/100
396/396 [==============================] - 0s 59us/step - loss: 1.1798 - mean_squared_error: 1.1798 - val_loss: 0.3197 - val_mean_squared_error: 0.3197
Epoch 55/100
396/396 [==============================] - 0s 51us/step - loss: 1.1056 - mean_squared_error: 1.1056 - val_loss: 0.2804 - val_mean_squared_error: 0.2804
Epoch 56/100
396/396 [=

Epoch 100/100
396/396 [==============================] - 0s 49us/step - loss: 1.0309 - mean_squared_error: 1.0309 - val_loss: 0.3295 - val_mean_squared_error: 0.3295


Finally, run the cell below to calcluate the MSE for our final version of this model and see how a learning rate decay strategy affected the model. 

In [52]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [53]:
print(MSE_train) 
print(MSE_val) 

0.99123871467396
0.3294581818726523


## Further reading

https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network